In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [2]:
(x_train, y_train), (x_test,y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [11]:
features = ['CRIM', 'ZN', 
            'INDUS','CHAS','NOX','RM','AGE',
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
x_train_df = pd.DataFrame(x_train,columns=features)
x_test_df = pd.DataFrame(x_test, columns=features)
y_train_df = pd.DataFrame(y_train,columns=['MEDV'])
y_test_df = pd.DataFrame(y_test,columns=['MEDV'])
x_train_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [0]:
# To transform the train and test DataFrame (pandas) to tensorflow dataset  (This is done so we can use feature_columns to map from the columns in the Pandas dataframe to features used to train the model)

def estimator_input_fn(df_data,df_label,epochs=10,shuffle=True,batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(df_data),df_label))
    if shuffle:
      ds = ds.shuffle(100)    #Randomly shuffle the dataset 
    ds = ds.batch(batch_size).repeat(epochs)     #Makes ds a dataset with a specified batch size (No. of elements in the dataset) and repeats this whole dataset 'epochs' number of times
    return ds 
  return input_function

 #Convert from a pandas dataframe to tensorflow dataset 
train_input_fn = estimator_input_fn(x_train_df,y_train_df)   
val_input_fn = estimator_input_fn(x_test_df, y_test_df,epochs=1,shuffle=False)

In [0]:
 #Feature_columns define how the input data is intepreted by the model. Output of the feature_columns is the input used by the model

feature_columns = []  
for feature_names in features:
  feature_columns.append(fc.numeric_column(feature_names,dtype=tf.float32))   #Since the input features are all numbers, use .numeric_column

In [24]:
# use an estimator which is basically a pre-defined model. It covers training, evaluation, prediction and exporting. Dont have to implement the details for yourself (e.g metrics, activation function, type of layers,etc..)

linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)
linear_est.train(train_input_fn,steps=100)
result = linear_est.evaluate(val_input_fn)
print(result)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbsr83hd8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. 

In [26]:
result = linear_est.predict(val_input_fn)
for pred, exp in zip(result,y_test[:32]):
  print(f'Predicted Value: {pred["predictions"][0]}. Expected value: {exp}')

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbsr83hd8/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted Value: 3.716794729232788. Expected value: 7.2
Predicted Value: 23.082910537719727. Expected value: 18.8
Predicted Value: 21.55373191833496. Expected value: 19.0
Predicted Value: 22.101890563964844. Expected value: 27.0
Predicted Va